In [1]:
import re

In [2]:
# !head /mnt/data/group3/wiki-play/data/wiki/instance-types_specific.ttl

In [3]:
# entities_dict={}
# with open("/mnt/data/group3/wiki-play/data/wiki/instance-types_specific.ttl") as input_file:
#     for index ,element in enumerate(input_file):
#         elements = element.strip().split(" ")
# #         print(elements)
#         el_1 = elements[0].replace("http://wikidata.dbpedia.org/resource/","")[1:-1]
#         el_2 = elements[2].replace("http://dbpedia.org/ontology/","")[1:-1]
#         entities_dict[el_1] = el_2
#         print(el_1,el_2)
        

In [4]:
# !pip install pandas

In [5]:
# !head /mnt/data/group3/wiki-play/data/wiki/instance-types_specific.ttl

In [6]:
import ray
ray.shutdown()
ray.init(num_cpus=16)
import modin.pandas as pd

2020-06-01 14:24:47,825	INFO resource_spec.py:212 -- Starting Ray with 327.88 GiB memory available for workers and up to 144.52 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-01 14:24:48,144	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


In [55]:
!ls /mnt/data/group3/wiki-play/data/wiki/*.ttl

 /mnt/data/group3/wiki-play/data/wiki/instance-types_specific.ttl
 /mnt/data/group3/wiki-play/data/wiki/instance-types_transitive.ttl
'/mnt/data/group3/wiki-play/data/wiki/page_lang=en_ids.ttl'


In [57]:
!head /mnt/data/group3/wiki-play/data/wiki/instance-types_transitive.ttl

# started 2020-05-16T14:51:48Z
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Country> .
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.wikidata.org/entity/Q6256> .
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/PopulatedPlace> .
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Place> .
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Place> .
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Location> .
<http://wikidata.dbpedia.org/resource/Q17> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Thing> .
<http://wikidata.dbpedia.org/resource/Q20> <http://ww

In [7]:
df = pd.read_csv("/mnt/data/group3/wiki-play/data/wiki/instance-types_specific.ttl", 
                 skiprows=1,
                 header=None, 
                 delim_whitespace=True)


In [8]:
df.head()

,0,1,2,3
0,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
1,<http://wikidata.dbpedia.org/resource/Q20>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
2,<http://wikidata.dbpedia.org/resource/Q25>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
3,<http://wikidata.dbpedia.org/resource/Q27>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
4,<http://wikidata.dbpedia.org/resource/Q30>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.


In [9]:
df.columns = ["wikiId", "type", "fixedClass", "dot"]
df.head()

,wikiId,type,fixedClass,dot
0,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
1,<http://wikidata.dbpedia.org/resource/Q20>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
2,<http://wikidata.dbpedia.org/resource/Q25>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
3,<http://wikidata.dbpedia.org/resource/Q27>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.
4,<http://wikidata.dbpedia.org/resource/Q30>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.


In [10]:
# df[df[0]=="<http://wikidata.dbpedia.org/resource/Q17>"][2].values.tolist()

In [11]:
import urllib.parse

list_of_urls= []
def preprocess(element):
    element = re.search("\"(.*?)\"", element.strip("\n"))
    element = element.group()[1:-1]
    return element
# with open("../data/wiki/dbqueries.txt","r") as input_file:
#     for element in input_file:
#         element = re.search("\"(.*?)\"", element.strip("\n"))
#         element = element.group()[1:-1]
#         list_of_urls.append(element)

In [12]:
df_url = pd.read_csv("../data/wiki/dbqueries.txt",header=None)


In [13]:
df_url.head()

,0
0,"./AA/wiki_00:<a href=""12-bit"">"
1,"./AA/wiki_00:<a href=""12th%20meridian%20east"">"
2,"./AA/wiki_00:<a href=""16-bit"">"
3,"./AA/wiki_00:<a href=""1848%20United%20States%2..."
4,"./AA/wiki_00:<a href=""1856%20Republican%20Nati..."


In [14]:
df_url.columns = ["wikiTitle"]
df_url["wikiTitle"]= df_url["wikiTitle"].apply(preprocess) 
df_url.head()


,wikiTitle
0,12-bit
1,12th%20meridian%20east
2,16-bit
3,1848%20United%20States%20presidential%20election
4,1856%20Republican%20National%20Convention


In [15]:
df_url.count

<bound method BasePandasDataset.count of                                                  wikiTitle
0                                                   12-bit
1                                   12th%20meridian%20east
2                                                   16-bit
3         1848%20United%20States%20presidential%20election
4                1856%20Republican%20National%20Convention
...                                                    ...
80470555                                           Zionism
80470556                                           Zionist
80470557                                      Zone%20plate
80470558                           Zoutpansberg%20Commando
80470559                      Zwettl-Nieder%C3%B6sterreich

[80470560 rows x 1 columns]>

In [16]:
df_url["wikiTitle"]= df_url["wikiTitle"].apply(lambda x:urllib.parse.unquote(x)) 

In [17]:
# df_url.wikiTitle.drop_duplicates(inplace=True)
df_url.head()

,wikiTitle
0,12-bit
1,12th meridian east
2,16-bit
3,1848 United States presidential election
4,1856 Republican National Convention


In [18]:
df_url.count

<bound method BasePandasDataset.count of                                          wikiTitle
0                                           12-bit
1                               12th meridian east
2                                           16-bit
3         1848 United States presidential election
4              1856 Republican National Convention
...                                            ...
80470555                                   Zionism
80470556                                   Zionist
80470557                                Zone plate
80470558                     Zoutpansberg Commando
80470559                   Zwettl-Niederösterreich

[80470560 rows x 1 columns]>

In [33]:
df_url.wikiTitle.nunique()

8364111

In [19]:
from wikimapper import WikiMapper

mapper = WikiMapper("wikimapper/data/index_enwiki-20190420.db")


In [20]:
df_url["wikiId"]= df_url["wikiTitle"].apply(lambda x: mapper.title_to_id(x.replace(" ","_")))

In [21]:
# df_url[df_url["wikiId"].isnull()].count()

In [22]:
df_url.head()

KeyboardInterrupt: 

In [ ]:
80470560-30458743

In [23]:
df_url["wikiId"]= df_url["wikiId"].apply(lambda x: None if not x else "<http://wikidata.dbpedia.org/resource/"+x+">") 

In [ ]:
df_url.iloc[0]

In [25]:
tmp =pd.merge(df,df_url, on="wikiId", how='inner')

To request implementation, send an email to feature_requests@modin.org.


In [ ]:
# new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])

In [32]:
tmp.wikiTitle.nunique()

2725490

In [ ]:
# not_found=0
# found =0
# not_in_mapper =0
# for i in list_of_urls:
#     id = mapper.title_to_id(i.replace("%20","_"))
# #     print(id,i)

#     if id:
#         if df[df[0]=="<http://wikidata.dbpedia.org/resource/"+id+">"][2].values.tolist():
# #             print(id,entities_dict["http://wikidata.dbpedia.org/resource/"+id])
#             found+=1
#         else:
#             not_found+=1
# #             print("not in dict")
#     else:
#         not_in_mapper+=1
# print(not_found,found,not_in_mapper)

In [ ]:
# https://www.urldecoder.io/python/

In [27]:
tmp.count()

wikiId        29229349
type          29229349
fixedClass    29229349
dot           29229349
wikiTitle     29229349
dtype: int64

In [34]:
tmp2 =pd.merge(df,df_url, on="wikiId", how='right')

In [35]:
tmp2.head()

,wikiId,type,fixedClass,dot,wikiTitle
0,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.,Japan
1,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.,Japan
2,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.,Japan
3,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.,Japan
4,<http://wikidata.dbpedia.org/resource/Q17>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Country>,.,Japan


In [40]:
result = tmp.drop_duplicates()

In [51]:
len(result)

2741974

In [60]:
result[["fixedClass","wikiTitle","wikiId"]].to_csv("de-duplicated_result.csv",index=False)

In [61]:
!head de-duplicated_result.csv

fixedClass,wikiTitle,wikiId
<http://dbpedia.org/ontology/Country>,Japan,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,State of Japan,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,Nihon,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,JAPAN,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,日本,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,Japan (country),<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,Nippon,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,JPN,<http://wikidata.dbpedia.org/resource/Q17>
<http://dbpedia.org/ontology/Country>,Norway,<http://wikidata.dbpedia.org/resource/Q20>


In [ ]:
# http://downloads.dbpedia.org/repo/dbpedia/wikidata/instance-types/2020.05.01/